In [1]:
!pip install python-docx
!pip install PyMuPDF
!pip install pillow


In [21]:
import docx
import fitz
import pytesseract
from PIL import Image
import re

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
def ocr_image(image):
    return pytesseract.image_to_string(image, lang='eng')

def clean_text(text):
    # Remove extra line breaks and multiple spaces
    cleaned_text = re.sub(r'\n+', ' ', text)
    cleaned_text = re.sub(r' +', ' ', cleaned_text)

    # Remove page breaks
    cleaned_text = cleaned_text.replace('\x0c', '')

    # Add a line break before each point (number followed by a full stop)
    cleaned_text = re.sub(r'(\d+\.)', r'\n\1', cleaned_text)

    return cleaned_text.strip()

def scanned_pdf_to_word(pdf_path, output_path):
    try:
        # Create a new Word document
        doc = docx.Document()

        # Open the scanned PDF file using PyMuPDF (fitz)
        with fitz.open(pdf_path) as pdf_document:
            # Initialize a variable to store the content of all pages as a single paragraph
            full_text = ""

            # Process each page
            for page_num in range(pdf_document.page_count):
                page = pdf_document[page_num]

                # Render the page as an image
                image = page.get_pixmap()
                img = Image.frombytes("RGB", [image.width, image.height], image.samples)

                # Perform OCR on the image to extract text
                text = ocr_image(img)

                # Clean the extracted text
                cleaned_text = clean_text(text)

                # Append the cleaned text to the full_text variable
                full_text += cleaned_text + " "

        # Add the full text as a single paragraph in the Word document
        doc.add_paragraph(full_text)
        
        # Save the Word document to the specified output path
        doc.save(output_path)

        print(f"Scanned PDF converted to Word and saved to: {output_path}")
    except Exception as e:
        print(f"Error while converting the scanned PDF: {e}")

# Replace 'path/to/your/scanned_document.pdf' with the actual path of your scanned PDF
pdf_path = 'Petition-1.pdf'

# Replace 'path/to/your/output/document.docx' with the desired output path for the Word document
output_path = 'output-2.docx'

scanned_pdf_to_word(pdf_path, output_path)



Scanned PDF converted to Word and saved to: output-2.docx
